# Example of Kan-Derived Function fb

In [1]:
import os
from kan import *
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
import numpy as np
import torch
import HydroErr as he

# Input

In [2]:
os.chdir('/Users/chuyangliu/Documents/Postdoc/Cive898/Writtings/GitHub_kan4baseflow')
os.getcwd()

'/Users/chuyangliu/Documents/Postdoc/Cive898/Writtings/GitHub_kan4baseflow'

In [3]:
col_index = 'Qb_hat/P_hat_true'
dir_in = r'./Datasets/Meira Neto et al 2020.xlsx' 

# Helper functions

In [4]:
def to_numpy_1d(array_like):
    import torch
    if isinstance(array_like, torch.Tensor):
        array_like = array_like.detach().cpu().numpy()
    return np.ravel(array_like)

def model_performance(sim_train, obs_train, sim_test, obs_test, additional_metrics=None):
    import HydroErr as he
    sim_train = to_numpy_1d(sim_train)
    obs_train = to_numpy_1d(obs_train)
    sim_test = to_numpy_1d(sim_test)
    obs_test = to_numpy_1d(obs_test)

    metrics = {
        'R^2': he.r_squared,
        'RMSE': he.rmse,
        'KGE': he.kge_2012,
        'NSE': he.nse
    }

    if additional_metrics:
        metrics.update(additional_metrics)

    def evaluate_metrics(sim, obs):
        results = {}
        for name, func in metrics.items():
            results[name] = func(sim, obs)
        return results

    train_results = evaluate_metrics(sim_train, obs_train)
    test_results = evaluate_metrics(sim_test, obs_test)

    print(f"Model performance on training data:")
    for name, value in train_results.items():
        print(f"{name} (train): {value}")

    print(f"\nModel performance on test data:")
    for name, value in test_results.items():
        print(f"{name} (test): {value}")


In [5]:
def average_r2(df_all, opt, lamb, lamb_coef, grid, seed, lib, x_label='Phi', y_label='Qb_hat/P_hat_true', n_splits=10,
               random_state=0, verbose=0):
    
    df_all['Qd_hat/P_hat_true'] = df_all['Qd'] / df_all['P']
    df_all['Qb_hat/P_hat_true'] = df_all['Qb'] / df_all['P']

    X_all = df_all[[x_label]].copy()
    y_all = df_all[y_label].values.reshape(-1, 1).copy()

    X, X_test_all, y, y_test_all = train_test_split(X_all, y_all, test_size=0.2, random_state=random_state)


    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    list_r2_test = []

    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        print(f'\nFold {fold + 1}')
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        n_repeat = 1
        X_train = np.repeat(X_train, n_repeat, axis=0)
        y_train = np.repeat(y_train, n_repeat, axis=0)
        X_test = X_test.values
        y_test = y_test.flatten()

        X_train = torch.from_numpy(X_train).float()
        X_test = torch.from_numpy(X_test).float()
        y_train = torch.from_numpy(y_train)
        y_test = torch.from_numpy(y_test)

        dataset = {
            'train_input': X_train,
            'test_input': X_test,
            'train_label': y_train,
            'test_label': y_test
        }

        model = KAN(width=[X_train.shape[1], 1], grid=grid, k=3, seed=seed)
        model(dataset['train_input'])

        model.train(dataset, opt=opt, steps=20, lamb=lamb, lamb_coef=lamb_coef)

        for epoch in range(2):
            results = model.train(dataset, opt=opt, steps=50, lamb=lamb, lamb_coef=lamb_coef)


        model.auto_symbolic(lib=lib)

        for epoch in range(1):
            model.train(dataset, opt=opt, steps=50, lamb=lamb, lamb_coef=lamb_coef)

        y_train_predicted = model(X_train)
        y_test_predicted = model(X_test)

        r2_train = he.r_squared(y_train_predicted.detach().numpy()[:, 0], y_train.detach().numpy().flatten())
        r2_test = he.r_squared(y_test_predicted.detach().numpy()[:, 0], y_test.detach().numpy().flatten())

        list_r2_test.append(r2_test)

        if verbose > 0:
            print(f"... KAN eqn: {model.symbolic_formula()[0][0]}")
            print(f'... Pre-training set (R^2): {r2_train}')
            print(f'... Validation set (R^2): {r2_test}')

        # -- all test
        y_pred_all = model(torch.from_numpy(X_test_all.values).float())
        r2_all = he.r_squared(y_pred_all.detach().numpy().flatten(), y_test_all.flatten())


        if verbose > 0:
            print(f"\n")
            model_performance(sim_train=model(torch.from_numpy(X.values).float()),
                              obs_train=y,
                              sim_test=y_pred_all,
                              obs_test=y_test_all,
                              additional_metrics=None)


    avg_r2 = np.nanmean(list_r2_test)


    return avg_r2

# Load Data


In [6]:
df_all = pd.read_excel(dir_in)

# Hyperparameter tuning

In [7]:
# Input
opt = "LBFGS"
lamb = 1e-2 
lamb_coef = 0.0
width = 1
lib = None

In [8]:
# -----   Grid search with 10-fold cross validation ----
# Example range of seeds and grid values to try
seed_range = range(0, 2)  
grid_range = range(2, 4)   

best_r2 = -float('inf')
best_seed = None
best_grid = None

# Iterate over different combinations of seed and grid
for seed in seed_range:
    for grid in grid_range:
        print(f"Testing seed={seed}, grid={grid}")

        avg_r2 = average_r2(df_all, opt, lamb, lamb_coef, grid, seed, lib,
                            x_label='Phi', y_label='Qb_hat/P_hat_true',
                            n_splits=10, random_state=5)

        print(f"Average R^2: {avg_r2}")

        # Update best parameters if current combination is better
        if avg_r2 > best_r2:
            best_r2 = avg_r2
            best_seed = seed
            best_grid = grid

print(f"\nBest parameters found:")
print(f"Seed: {best_seed}")
print(f"Grid: {best_grid}")
print(f"Best average R^2: {best_r2}")

Testing seed=0, grid=2

Fold 1


train loss: 6.16e-02 | test loss: 1.34e-01 | reg: 1.10e-02 : 100%|██| 20/20 [00:00<00:00, 27.93it/s]
train loss: 6.16e-02 | test loss: 1.34e-01 | reg: 1.10e-02 : 100%|██| 50/50 [00:00<00:00, 96.71it/s]
train loss: 6.16e-02 | test loss: 1.34e-01 | reg: 1.10e-02 : 100%|██| 50/50 [00:00<00:00, 89.87it/s]


fixing (0,0,0) with x^4, r2=0.9998977780342102


train loss: 6.48e-02 | test loss: 1.38e-01 | reg: 1.08e-02 : 100%|██| 50/50 [00:00<00:00, 80.95it/s]



Fold 2


...


Fold 10


train loss: 5.96e-02 | test loss: 1.65e-01 | reg: 1.10e-02 : 100%|██| 20/20 [00:00<00:00, 66.95it/s]
train loss: 5.96e-02 | test loss: 1.65e-01 | reg: 1.10e-02 : 100%|█| 50/50 [00:00<00:00, 118.78it/s]
train loss: 5.96e-02 | test loss: 1.65e-01 | reg: 1.10e-02 : 100%|█| 50/50 [00:00<00:00, 103.89it/s]


fixing (0,0,0) with x^3, r2=0.9999986290931702


train loss: 6.76e-02 | test loss: 1.53e-01 | reg: 9.95e-03 : 100%|█| 50/50 [00:00<00:00, 141.65it/s]

Average R^2: 0.6671218566198217

Best parameters found:
Seed: 0
Grid: 3
Best average R^2: 0.7454046508549044


# Check the optimal model with the best parameters

In [9]:
average_r2(df_all, opt, lamb, lamb_coef, best_grid, best_seed, lib,
           x_label='Phi', y_label='Qb_hat/P_hat_true',
           n_splits=10, random_state=5, verbose = 2)


Fold 1


train loss: 6.16e-02 | test loss: 1.34e-01 | reg: 1.11e-02 : 100%|██| 20/20 [00:00<00:00, 40.01it/s]
train loss: 6.16e-02 | test loss: 1.34e-01 | reg: 1.11e-02 : 100%|█| 50/50 [00:00<00:00, 130.11it/s]
train loss: 6.16e-02 | test loss: 1.34e-01 | reg: 1.11e-02 : 100%|█| 50/50 [00:00<00:00, 110.45it/s]


fixing (0,0,0) with x^3, r2=0.9999996423721313


train loss: 7.01e-02 | test loss: 1.37e-01 | reg: 1.01e-02 : 100%|█| 50/50 [00:00<00:00, 101.57it/s]


... KAN eqn: 0.52*(1 - 0.24*x_1)**3 - 0.e-2
... Pre-training set (R^2): 0.6639028293563374
... Validation set (R^2): 0.7493244706449997


Model performance on training data:
R^2 (train): 0.6706621257799479
RMSE (train): 0.0686342646624862
KGE (train): 0.7371960596143217
NSE (train): 0.670584567585866

Model performance on test data:
R^2 (test): 0.6590677133713186
RMSE (test): 0.06775534440622558
KGE (test): 0.6887054591594054
NSE (test): 0.6515954861586928

Fold 2


...

train loss: 5.96e-02 | test loss: 1.65e-01 | reg: 1.10e-02 : 100%|██| 20/20 [00:00<00:00, 37.74it/s]
train loss: 5.96e-02 | test loss: 1.65e-01 | reg: 1.10e-02 : 100%|█| 50/50 [00:00<00:00, 149.87it/s]
train loss: 5.96e-02 | test loss: 1.65e-01 | reg: 1.10e-02 : 100%|██| 50/50 [00:00<00:00, 94.90it/s]


fixing (0,0,0) with x^3, r2=0.9999986886978149


train loss: 1.10e-01 | test loss: 1.23e-01 | reg: 9.72e-03 : 100%|█| 50/50 [00:00<00:00, 114.37it/s]


... KAN eqn: 0.11*(1 - 0.4*x_1)**3 + 0.17
... Pre-training set (R^2): 0.14571679253217518
... Validation set (R^2): 0.7049082184211856


Model performance on training data:
R^2 (train): 0.15314886347845938
RMSE (train): 0.11009045973432595
KGE (train): 0.12055943591271734
NSE (train): 0.1524577800824073

Model performance on test data:
R^2 (test): 0.6915225088015506
RMSE (test): 0.10254940418025006
KGE (test): 0.11276289356104474
NSE (test): 0.20188965248469604


0.7454046508549044

# Optimal model and its model performance
'''
... KAN eqn: 0.39 - 0.34*tanh(1.42*x_1 - 0.82)

Model performance on training data:
R^2 (train): 0.7474905810222926
RMSE (train): 0.06010343033108799
KGE (train): 0.8040276374351344
NSE (train): 0.7473842117724503

Model performance on test data:
R^2 (test): 0.7440847811189126
RMSE (test): 0.058473655281330465
KGE (test): 0.7943953652440818
NSE (test): 0.7405120799817747
'''